## Aula 9 - Agregação com Pandas

### Agregação com Pandas

In [ ]:
import pandas as pd
import numpy as np

#### Exemplo de Group By com Pandas

In [ ]:
# Criando um dataframe
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5), 'data2' : np.random.randn(5)})
df

In [ ]:
# Criar um objeto de agrupamento da coluna data1 agrupando por key1
grouped = df['data1'].groupby(df['key1'])
# Veja que o resultado retornado é um objeto Pyhton SeriesGroupBy 
grouped

In [ ]:
# Aplicando a métrica Média (mean) sobre esse objeto de agrupamento
# Perceba que a média sobre o objeto SeriesGroupBy vai retornar uma Series como resultado,
#   e o índice será a coluna de agrupamento key1
grouped.mean()

#### Agregação com duas colunas

In [ ]:
# Criar um objeto de agrupamento da coluna data1 agrupando por key1 e key2
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
# Veja que means é uma Series com índices key1 e key2
means

In [ ]:
# Curiosidade: você pode pivotar essa série com o médoto unstack()
means.unstack()

#### Aplicando a métrica em todas as colunas de dados

In [ ]:
# Agrupe o dataframe df por key1 e extraia a média de todas as colunas
df.groupby('key1').mean()

In [ ]:
# perceba que o objeto retornado pelo agrupamento do dataframe inteiro é 
#   um DataFrameGroupBy 
df.groupby('key1')

In [ ]:
# Agrupe o dataframe inteiro pelas colunas key1 e key2 calculando o valor da média
#    para as outras colunas.
df.groupby(['key1', 'key2']).mean()

#### Mostrando o tamanho dos grupos

In [ ]:
# Mostre o tamanho dos grupos de um agrupamento do dataframe df pelas colunas key1 e key2
df.groupby(['key1', 'key2']).size()

#### Agrupando com uma função

In [ ]:
# Considere o seguinte dataframe
people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'], 
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Adicionando alguns valores NA
people

In [ ]:
# Agrupe o dataframe people pelo tamanho das strings do indice retornando a soma de cada
#    grupo.
people.groupby(len).sum()

#### Aplicando várias métricas ao mesmo tempo

In [ ]:
# Considere o seguinte dataset de gorgetas (tips)
import pandas as pd
import io # Biblioteca com funções de IO (dentre elas de Strings)
import requests #Biblioteca para fazer requisições HTTP
url="https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/tips.csv"
s=requests.get(url).content
tips=pd.read_csv(io.StringIO(s.decode('utf-8')))
tips.head()

In [ ]:
# Adiciona uma coluna para calcular a porcentage da gorjeta em relação ao valor da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

In [ ]:
# Agrupar por dia da semana (day) e fumante (smoker)
grouped = tips.groupby(['day', 'smoker'])
grouped

In [ ]:
# Pegar um slice (uma fatia) do objeto de agrupamento com apenas a coluna tip_pct
grouped_pct = grouped[['tip_pct', 'total_bill']]
grouped_pct

In [ ]:
# Calcular a média deste agrupamento
grouped_pct.agg('mean')

In [ ]:
# Considere a função para retornar o intervalo de variação de um array
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [ ]:
# Escreva numa só sentença a aplicação das métricas média, desvio padrão (std) 
#   e peak_to_peak ao slice do agrupamento criado acima
pk = lambda x: x.max() - x.min()
grouped_pct.agg({'total_bill': ['mean', pk]})

#### Várias métricas em diferentes colunas

In [ ]:
# Usando um dicionário para mapear as colunas e suas respectivas métricas, aplique
#     à coluna tip_pct as métricas 'min', 'max', 'mean', 'std' e 
dfr = grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})
dfr.reset_index()

### Variáveis Categóricas (Categorical)

In [ ]:
# Criando dados categóricos ordenados
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=["Seg", "Ter", "Qua", 
                        "Qui", "Sex", "Sab", "Dom"], ordered=True)

df = pd.DataFrame({'Bairro1': ['Dom', 'Sab', 'Dom', 'Qui', 'Dom', 'Sab'], 
                  'Bairro2': ['Sex', 'Sex', 'Sex', 'Seg', 'Ter', 'Qui']},
                  dtype=cat_type)
df

In [ ]:
df[df['Bairro1'] > 'Sab'].head()

### Resolvendo Problemas com o Pandas

#### Correlação entre a variação percentual de dois ativos financeiros. Exemplo: Ação do Walmart e da Amazon


In [ ]:
###### Instalação de bibliotecas #####
# Use o seguinte comando: !pip install <nome_biblioteca>
# Exemplo: !pip install yahooquery
!pip install yahooquery

In [ ]:
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from yahooquery import Ticker

In [ ]:
# Definir uma lista de simbolos (ações) de interesse
symbols_list = ['AMZN', 'WMT']
# Criar um objeto Ticker (para fazer o download de dados de cotação de ações)
tc = Ticker(symbols_list)
# Fazer o download de dados de um período de 60 dias, com intervalor de 1d entre cada medição
# Lista de períodos e intervalos possíveis: https://yahooquery.dpguthrie.com/guide/ticker/historical/
df = tc.history(period='5d', interval="1d")
# Mostrar os primeiros 5 registros do dataframe df
# df = df['close']
df.head()

In [ ]:
df_reset = df.reset_index()
df_reset['ontem'] = df_reset['date'].apply(lambda x: x + datetime.timedelta(days=1))
df_merge = df_reset.merge(right=df_reset, left_on=['symbol','date'], 
                          right_on=['symbol','ontem'], suffixes=["", "_desloc"])
df_merge['change_pct'] = (df_merge['close'] - df_merge['close_desloc']) / df_merge['close_desloc']
df_pivot = df_merge.pivot('date', 'symbol', 'change_pct')


In [ ]:
df_pivot

In [ ]:
df_pivot.corr()

In [ ]:
df_result = df[['close']].unstack(level=0).pct_change()
df_result

In [ ]:
df_result.corr()

#### Concatenar verticalmente vários arquivos da mesma fonte de dados

In [ ]:
!git clone "https://github.com/alexlopespereira/enapespcd2021.git"

In [ ]:
!ls ./enapespcd2021/data/originais/morbidade/desagregado/2020/   

In [ ]:
!head ./enapespcd2021/data/originais/morbidade/desagregado/2020/A102927189_28_143_208.csv

In [ ]:
import glob
filepaths = glob.glob("./enapespcd2021/data/originais/morbidade/desagregado/2020/*.csv")
df_lista = [pd.read_csv(f, encoding='iso8859-1', skiprows=3, sep=';', skipfooter=7, engine='python') for f in filepaths]
df_concat = pd.concat(df_lista, ignore_index=True)

In [ ]:
df_concat.head()

#### Criar multiplos registros de atributos multi-valorados

In [ ]:
data = {'actors': {0: 'Blanche Bayliss, William Courtenay, Chauncey Depew',
  1: 'Elizabeth Tait, John Tait, Norman Campbell, Bella Cola, Will Coyne, Sam Crewes, Jack Ennis, John Forde, Vera Linden, Mr. Marshall, Mr. McKenzie, Frank Mills, Ollie Wilson',
  2: 'Asta Nielsen, Valdemar Psilander, Gunnar Helsengreen, Emil Albes, Hugo Flink, Mary Hagen'},
 'director': {0: 'Alexander Black', 1: 'Charles Tait', 2: 'Urban Gad'},
 'imdb_title_id': {0: 'tt0000009', 1: 'tt0000574', 2: 'tt0001892'},
 'title': {0: 'Miss Jerry',
  1: 'The Story of the Kelly Gang',
  2: 'Den sorte drøm'}}
df = pd.DataFrame(data)
df

In [ ]:
col = 'actors'
merge_on='imdb_title_id'
de = df[[col, merge_on]]
de[col] = de[col].str.split(",") 
de = de.explode(col)
df_merged = de.merge(right=df, on=merge_on, how='left', suffixes=["", "_y"])
df_merged

In [ ]:
def explode_and_merge(df, col, merge_on='id', split_on=";"):
    """Transforma um atributo multivalorado em várias linhas de conteúdo atômico,
       uma linha pra cada um dos atributos separados por ponto e vírgula
    """
    # Cria um novo dataframe
    # assign cria uma coluna no dataframe. ** separa um dicionário {k, v} nos argumentos k, v
    df_exp = df[[col, merge_on]].assign(**{col: df[col].str.split(split_on)}).explode(col)
    df_merged = df_exp.merge(right=df, on=merge_on, how='left', suffixes=["", "_y"])
    del df_merged[col+"_y"]
    return df_merged

In [ ]:
dfe = explode_and_merge(df, col='actors', merge_on='imdb_title_id', split_on=",")
dfe.head()

In [ ]:
data_google = {'keyword': {0: 'receita federal', 1: 'receita federal', 2: 'carteira de trabalho digital', 3: 'carteira de trabalho digital', 4: 'carteira de trabalho digital', 5: 'policia federal', 6: 'policia federal', 7: 'policia federal', 8: 'auxilio emergencial', 9: 'auxilio emergencial' },
 'paginas': {0: 'declarar-meu-imposto-de-renda', 1: 'obter-copia-de-declaracao-enviada-a-receita-federal',
  2: 'obter-a-carteira-de-trabalho', 3: 'solicitar-o-seguro-desemprego', 4: 'obter-carteira-digital-de-transito',
  5: 'obter-passaporte-comum-para-brasileiro', 6:'emitir-certidao-de-antecedentes-criminais', 7: 'adquirir-arma-de-fogo', 
  8: 'devolver-o-auxilio-emergencial-coronavirus-covid-19', 9:'contestar-o-resultado-do-auxilio-emergencial-coronavirus-covid-19'}}
df = pd.DataFrame(data_google)
posicao_grupo = df.groupby('keyword').cumcount()
df_google = pd.concat([df, posicao_grupo.rename('rank')], axis=1)

In [ ]:
df_google

In [ ]:
data_govbr = {'keyword': {0: 'receita federal', 1: 'receita federal', 2: 'carteira de trabalho digital', 3: 'carteira de trabalho digital', 4: 'carteira de trabalho digital', 5: 'policia federal', 6: 'policia federal', 7: 'policia federal', 8: 'auxilio emergencial', 9: 'auxilio emergencial' },
 'paginas': {
  0: 'consultar-comunicacoes-e-intimacoes-eletronicas-da-receita-federal', 1: 'consultar-cobrancas-da-receita-federal',
  2: 'obter-carteira-digital-de-transito', 3: 'obter-a-carteira-de-trabalho', 4: 'solicitar-o-seguro-desemprego', 
  5: 'emitir-certidao-de-antecedentes-criminais', 6:'adquirir-arma-de-fogo', 7: 'obter-passaporte-comum-para-brasileiro', 
  8: 'devolver-o-auxilio-emergencial-coronavirus-covid-19', 9: 'receber-auxilio-medico-emergencial'}}

df2 = pd.DataFrame(data_govbr)
posicao_grupo2 = df2.groupby('keyword').cumcount()
df_govbr = pd.concat([df2, posicao_grupo2.rename('rank')], axis=1)
df_govbr

In [ ]:
df_merge = df_govbr.merge(df_google, on=['keyword', 'values'], 
                          suffixes=['_govbr', '_google'])
df_merge['dif'] = (df_merge['rank_govbr'] - df_merge['rank_google']).abs()
print(df_merge['dif'].sum())

In [ ]:
df_merge

In [ ]:
data_govbr = {'keyword': {0: 'receita federal', 1: 'receita federal', 2: 'carteira de trabalho digital', 3: 'carteira de trabalho digital', 4: 'carteira de trabalho digital', 5: 'policia federal', 6: 'policia federal', 7: 'policia federal', 8: 'auxilio emergencial', 9: 'auxilio emergencial' },
 'values': {
  0: 'declarar-meu-imposto-de-renda', 1: 'obter-copia-de-declaracao-enviada-a-receita-federal',
  2: 'obter-a-carteira-de-trabalho', 3: 'solicitar-o-seguro-desemprego', 4: 'obter-carteira-digital-de-transito',
  5: 'obter-passaporte-comum-para-brasileiro', 6:'emitir-certidao-de-antecedentes-criminais', 7: 'adquirir-arma-de-fogo', 
  8: 'devolver-o-auxilio-emergencial-coronavirus-covid-19', 9:'contestar-o-resultado-do-auxilio-emergencial-coronavirus-covid-19'}}

df2 = pd.DataFrame(data_govbr)
df2

In [ ]:
data_govbr = {'keyword': {0: 'receita federal', 1: 'receita federal', 2: 'carteira de trabalho digital', 3: 'carteira de trabalho digital', 4: 'carteira de trabalho digital', 5: 'policia federal', 6: 'policia federal', 7: 'policia federal', 8: 'auxilio emergencial', 9: 'auxilio emergencial' },
 'values': {
  0: 'consultar-comunicacoes-e-intimacoes-eletronicas-da-receita-federal', 1: 'consultar-cobrancas-da-receita-federal',
  2: 'obter-carteira-digital-de-transito', 3: 'obter-a-carteira-de-trabalho', 4: 'solicitar-o-seguro-desemprego', 
  5: 'emitir-certidao-de-antecedentes-criminais', 6:'adquirir-arma-de-fogo', 7: 'obter-passaporte-comum-para-brasileiro', 
  8: 'devolver-o-auxilio-emergencial-coronavirus-covid-19', 9: 'receber-auxilio-medico-emergencial'}}

df3 = pd.DataFrame(data_govbr)
df3